\begin{center}
Gabe Morris
\end{center}

In [1]:
# Notebook Preamble
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('maroon_ipynb.mplstyle')

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 4403 & Homework 4 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 6-1
## Given
A 10-mm steel drill rod was heat treated and ground. The measured hardness was found to be 300 Brinell.

## Find
Estimate the endurance strength in $MPa$ if the rod is used in rotating bending.

## Solution
Eq. 6-10 on p. 305, 

$$S^\prime_e=\begin{cases}
0.5S_{ut} & S_{ut} \le 200\ ksi\ (1400\ MPa)\\
100 & S_{ut} > 200\ ksi \\
700\ MPa & S_{ut} > 1400\ MPa
\end{cases}$$

The ultimate strength of steel comes from Eq. 2-36,

$$S_{ut}=3.4H_B$$

In [2]:
H_B = 300
S_ut = sp.S('3.4')*H_B

if S_ut <= 1400:
    S_e_prime = 0.5*S_ut
else:
    S_e_prime = sp.S(700)

S_e_prime  # ksi

510.000000000000

This value is not the final value. The relationship for the refined value is,

$$S_e=k_ak_bk_ck_dk_eS^\prime_e$$

The only necessary $k$ values used for this analysis is $k_a$ and $k_b$, whose equations are at 6-18 and 6-19 respectfully.

In [3]:
# See Table 6-2
k_a = sp.S('1.58')*S_ut**-(sp.S('0.085'))
d = 10
k_b = (d/sp.S('7.62'))**-(sp.S('0.107'))
S_e = k_a*k_b*S_e_prime
S_e  # MPa

434.375608564977

The answer in the back of the book appears to be heavily rounded.